In [1]:
# only run if you are developping/debugging
# !pip install line_profiler
%load_ext line_profiler
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path

import pandas as pd
import random

from collections import Counter

from compgen2 import Gov, Matcher, GovTestData, Preprocessing
from compgen2.const import FILENAME_VL, FILENAME_GOV_TEST_SET
from compgen2.testdata import sample_test_set_from_gov, Synthetic, get_accuracy

#random.seed(1337)

In [3]:
data_root= Path("../data")

## Test sets
We use 4 test sets:
- different sample sets from the file "deutsche-verlustlisten-1wk"
- different sample sets from the gov database 
- different sample sets from a synthetic data set that tries to mimic the errors found in the original "verlustliste"
- manually collected correction suggestions from http://wiki-de.genealogy.net/Verlustlisten_Erster_Weltkrieg/Projekt/Ortsnamen

All test sets change when preprocessing is applied.

**Note**: We have a ground truth for all test setsbut the first one as the "verlustliste" is the actual problem we want to solve. So for the test set "verlustliste" we cannot calculate an accuracy score but we can compare who many items we were able to match. Assumption is that more matches are, in general, better.

## Test suite without Preprocessing -> Baseline

In [4]:
final_results = []

In [5]:
gov = Gov(data_root)
gov.load_data()
gov.build_indices()

In [14]:
# Test Set 1: VL
assert data_root.joinpath(FILENAME_VL).exists()
test_set_size = 200

vl = pd.read_parquet(data_root / FILENAME_VL)  # location column has the test data, truth is unknown

vl_test_sets = []
vl_test_sets.append(("vl test set with loc_count=1", vl.query("loc_parts_count == 1").sample(test_set_size)))
vl_test_sets.append(("vl test set with loc_count=2", vl.query("loc_parts_count == 2").sample(test_set_size)))
vl_test_sets.append(("vl test set with loc_count=3", vl.query("loc_parts_count == 3").sample(test_set_size)))
vl_test_sets.append(("vl test set containing '.'", vl[vl.location.str.contains(".", regex=False)].sample(test_set_size)))

In [7]:
# Test Set 2: Gov database
test_set_size = 100

gov_test_sets = []
gov_test_sets.append(("gov db test set with loc_count=1 and valid=1", sample_test_set_from_gov(gov, size=test_set_size, num_parts=1, valid=1)))
gov_test_sets.append(("gov db test set with loc_count=1 and valid=0.7", sample_test_set_from_gov(gov, size=test_set_size, num_parts=1, valid=0.7)))
gov_test_sets.append(("gov db test set with loc_count=2 and valid=1", sample_test_set_from_gov(gov, size=test_set_size, num_parts=2, valid=1)))
gov_test_sets.append(("gov db test set with loc_count=2 and valid=0.7", sample_test_set_from_gov(gov, size=test_set_size, num_parts=2, valid=0.7)))
gov_test_sets.append(("gov db test set with loc_count=3 and valid=1", sample_test_set_from_gov(gov, size=test_set_size, num_parts=3, valid=1)))
gov_test_sets.append(("gov db test set with loc_count=3 and valid=0.7", sample_test_set_from_gov(gov, size=test_set_size, num_parts=3, valid=0.7)))

In [8]:
# Test Set 3: Synthetic
test_set_size = 100

syn = Synthetic(gov)

syn_test_sets = []
syn_test_sets.append(("syn test set with default probabilities", syn.create_synthetic_test_set(test_set_size)))

In [9]:
# Test Set 4: GovTestData
assert data_root.joinpath(FILENAME_GOV_TEST_SET).exists()

gtd = GovTestData(gov)
gtd_test_sets = []
gtd_test_sets.append(("gov web test set", gtd.get_test_set()))

### Run the tests

In [10]:
result_row = []

Test Set 1

In [15]:
for name, test_set in vl_test_sets:
    m = Matcher(gov)
    print("Running", name)
    m.get_match_for_locations(test_set.location)
    total_matches = len([match for match in m.results.values() if match.get("possible_matches")])
    print(f"Total matches: {total_matches} ({round(total_matches / test_set.location.nunique() * 100, 4)}%).")
    print(Counter(m.matching_method))
    print()
    
    result_row.append(total_matches / test_set.location.nunique())

Running vl test set with loc_count=1


Processing locations: 100%|█████████████████████████████████████████████████████████████████████████████| 200/200 [06:05<00:00,  1.83s/it]


Total matches: 145 (72.5%).
Counter({'No anchor at all': 55, 'KREISORSTADT': 49, 'gov only': 44, 'Phonetic': 40, 'Cost 2': 3, 'Cost 5': 3, 'Cost 1': 3, 'Cost 4': 2, 'Cost 3': 1})

Running vl test set with loc_count=2


Processing locations: 100%|█████████████████████████████████████████████████████████████████████████████| 200/200 [00:16<00:00, 12.07it/s]


Total matches: 173 (86.5%).
Counter({'gov only': 175, 'KREISORSTADT': 12, 'Phonetic': 10, 'Cost 3': 1, 'Cost 5': 1, 'Cost 2': 1})

Running vl test set with loc_count=3


Processing locations: 100%|█████████████████████████████████████████████████████████████████████████████| 200/200 [00:13<00:00, 15.12it/s]


Total matches: 82 (41.0%).
Counter({'gov only': 186, 'KREISORSTADT': 12, 'Phonetic': 2})

Running vl test set containing '.'


Processing locations: 100%|█████████████████████████████████████████████████████████████████████████████| 200/200 [02:11<00:00,  1.52it/s]

Total matches: 174 (87.0%).
Counter({'gov only': 125, 'KREISORSTADT': 34, 'Phonetic': 20, 'No anchor at all': 18, 'Cost 5': 1, 'Cost 3': 1, 'Cost 4': 1})



Test Set 2

In [12]:
for name, test_set in gov_test_sets:
    m = Matcher(gov)
    print("Running", name)
    m.get_match_for_locations(test_set.location)
    total_matches = len([match for match in m.results.values() if match.get("possible_matches")])
    print(f"Total matches: {total_matches} ({round(total_matches / test_set.location.nunique() * 100, 4)}%).")
    
    accuracy = get_accuracy(m.results, test_set)
    print("Accuracy (entries where all parts of truth are in possible matches):", round(accuracy, 4))
    print()
    
    result_row.append(total_matches / test_set.location.nunique())
    result_row.append(accuracy)

Running gov db test set with loc_count=1 and valid=1


Processing locations: 100%|████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 46015.40it/s]

Total matches: 998 (99.8%).
Accuracy (entries where all parts of truth are in possible matches): 0.998



Running gov db test set with loc_count=1 and valid=0.7


Processing locations: 100%|████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 49200.05it/s]

...

Running gov db test set with loc_count=3 and valid=1


Processing locations: 100%|███████████████████████████████████████████████████████████████████████████| 1000/1000 [00:16<00:00, 60.27it/s]


Total matches: 994 (99.4%).
Accuracy (entries where all parts of truth are in possible matches): 0.991

Running gov db test set with loc_count=3 and valid=0.7


Processing locations: 100%|██████████████████████████████████████████████████████████████████████████| 1000/1000 [00:01<00:00, 771.78it/s]

Total matches: 696 (69.6%).
Accuracy (entries where all parts of truth are in possible matches): 0.696



Test Set 3

In [13]:
for name, test_set in syn_test_sets:
    m = Matcher(gov)
    print("Running", name)
    m.get_match_for_locations(test_set.location)
    total_matches = len([match for match in m.results.values() if match.get("possible_matches")])
    print(f"Total matches: {total_matches} ({round(total_matches /  test_set.location.nunique() * 100, 4)}%).")
    
    accuracy = get_accuracy(m.results, test_set)
    print("Accuracy (entries where all parts of truth are in possible matches):", round(accuracy, 4))
    print()
    
    result_row.append(total_matches / test_set.location.nunique())
    result_row.append(accuracy)

Running syn test set with default probabilities


Processing locations:  26%|███████████████████▉                                                        | 262/1000 [00:56<02:39,  4.61it/s]


KeyboardInterrupt: 

Test Set 4

In [14]:
for name, test_set in gtd_test_sets:
    m = Matcher(gov)
    print("Running", name)
    m.get_match_for_locations(test_set.location)
    total_matches = len([match for match in m.results.values() if match.get("possible_matches")])
    print(f"Total matches: {total_matches} ({round(total_matches /  test_set.location.nunique() * 100, 4)}%).")
    
    accuracy = get_accuracy(m.results, test_set)
    print("Accuracy (entries where all parts of truth are in possible matches):", round(accuracy, 4))
    print()
    
    result_row.append(total_matches / test_set.location.nunique())
    result_row.append(accuracy)

Running gov web test set


Processing locations: 100%|████████████████████████████████████████████████████████| 5641/5641 [06:43<00:00, 13.98it/s]

Total matches: 5029 (90.1739%).
Accuracy (entries where all parts of truth are in possible matches): 0.6462



In [15]:
final_results.append(result_row)

## Test suite with VL + Gov Preprocessing - replace corrections and characters, no substitution

In [16]:
result_row = []

In [17]:
for _, test_set in vl_test_sets + gov_test_sets + syn_test_sets + gtd_test_sets:
    test_set.location = Preprocessing.replace_corrections_vl(test_set.location)
    test_set.location = Preprocessing.replace_characters_vl(test_set.location)
    
    if "truth" in test_set:
        test_set.truth = Preprocessing.replace_corrections_vl(test_set.truth)
        test_set.truth = Preprocessing.replace_characters_vl(test_set.truth)

In [18]:
gov = Gov(data_root)
gov.load_data()
old_names = list(gov.ids_by_name.keys())
new_names = Preprocessing.replace_characters_gov(pd.Series(old_names, dtype=str))

for old_name, new_name in zip(old_names, new_names):
    gov.ids_by_name[new_name] = gov.ids_by_name[old_name]
    del gov.ids_by_name[old_name]
    
for id_ in gov.names_by_id:
    names = gov.names_by_id[id_]
    gov.names_by_id[id_] =  Preprocessing.replace_characters_gov(pd.Series(names, dtype=str))
    
gov.build_indices()

Test Set 1

In [19]:
for name, test_set in vl_test_sets:
    m = Matcher(gov)
    print("Running", name)
    m.get_match_for_locations(test_set.location)
    total_matches = len([match for match in m.results.values() if match.get("possible_matches")])
    print(f"Total matches: {total_matches} ({round(total_matches / test_set.location.nunique() * 100, 4)}%).")
    print()
    
    result_row.append(total_matches / test_set.location.nunique())

Running vl test set with loc_count=1


Processing locations: 100%|████████████████████████████████████████████████████████| 1000/1000 [15:11<00:00,  1.10it/s]


Total matches: 834 (83.4%).

Running vl test set with loc_count=2


Processing locations: 100%|████████████████████████████████████████████████████████| 1000/1000 [00:47<00:00, 21.26it/s]


Total matches: 853 (85.3%).

Running vl test set with loc_count=3


Processing locations: 100%|████████████████████████████████████████████████████████| 1000/1000 [00:46<00:00, 21.64it/s]


Total matches: 500 (50.0%).

Running vl test set containing '.'


Processing locations: 100%|████████████████████████████████████████████████████████| 1000/1000 [07:33<00:00,  2.20it/s]

Total matches: 890 (89.0%).



Test Set 2

In [20]:
for name, test_set in gov_test_sets:
    m = Matcher(gov)
    print("Running", name)
    m.get_match_for_locations(test_set.location)
    total_matches = len([match for match in m.results.values() if match.get("possible_matches")])
    print(f"Total matches: {total_matches} ({round(total_matches / test_set.location.nunique() * 100, 4)}%).")
    
    accuracy = get_accuracy(m.results, test_set)
    print("Accuracy (entries where all parts of truth are in possible matches):", round(accuracy, 4))
    print()
    
    result_row.append(total_matches / test_set.location.nunique())
    result_row.append(accuracy)

Running gov db test set with loc_count=1 and valid=1


Processing locations: 100%|█████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 37038.41it/s]


Total matches: 998 (99.8%).
Accuracy (entries where all parts of truth are in possible matches): 0.996

Running gov db test set with loc_count=1 and valid=0.7


Processing locations: 100%|█████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 31243.19it/s]


Total matches: 698 (69.8%).
Accuracy (entries where all parts of truth are in possible matches): 0.695

Running gov db test set with loc_count=2 and valid=1


...

Total matches: 691 (69.1%).
Accuracy (entries where all parts of truth are in possible matches): 0.685

Running gov db test set with loc_count=3 and valid=1


Processing locations: 100%|███████████████████████████████████████████████████████| 1000/1000 [00:07<00:00, 128.81it/s]


Total matches: 974 (97.4%).
Accuracy (entries where all parts of truth are in possible matches): 0.97

Running gov db test set with loc_count=3 and valid=0.7


Processing locations: 100%|███████████████████████████████████████████████████████| 1000/1000 [00:01<00:00, 676.86it/s]

Total matches: 687 (68.7%).
Accuracy (entries where all parts of truth are in possible matches): 0.684



Test Set 3

In [21]:
for name, test_set in syn_test_sets:
    m = Matcher(gov)
    print("Running", name)
    m.get_match_for_locations(test_set.location)
    total_matches = len([match for match in m.results.values() if match.get("possible_matches")])
    print(f"Total matches: {total_matches} ({round(total_matches /  test_set.location.nunique() * 100, 4)}%).")
    
    accuracy = get_accuracy(m.results, test_set)
    print("Accuracy (entries where all parts of truth are in possible matches):", round(accuracy, 4))
    print()
    
    result_row.append(total_matches / test_set.location.nunique())
    result_row.append(accuracy)

Running syn test set with default probabilities


Processing locations: 100%|████████████████████████████████████████████████████████| 1000/1000 [02:06<00:00,  7.90it/s]

Total matches: 971 (97.9818%).
Accuracy (entries where all parts of truth are in possible matches): 0.806



Test Set 4

In [ ]:
for name, test_set in gtd_test_sets:
    m = Matcher(gov)
    print("Running", name)
    m.get_match_for_locations(test_set.location)
    total_matches = len([match for match in m.results.values() if match.get("possible_matches")])
    print(f"Total matches: {total_matches} ({round(total_matches /  test_set.location.nunique() * 100, 4)}%).")
    
    accuracy = get_accuracy(m.results, test_set)
    print("Accuracy (entries where all parts of truth are in possible matches):", round(accuracy, 4))
    print()
    
    result_row.append(total_matches / test_set.location.nunique())
    result_row.append(accuracy)

Running gov web test set


Processing locations:  59%|█████████████████████████████████▏                      | 3349/5641 [03:58<02:53, 13.23it/s]

In [ ]:
final_results.append(result_row)

## Test suite with VL + Gov Preprocessing - replace corrections and characters + substitution

In [ ]:
result_row = []

In [ ]:
for _, test_set in vl_test_sets + gov_test_sets + syn_test_sets + gtd_test_sets:
    test_set.location = Preprocessing.substitute_partial_words(test_set.location, data_root)
    test_set.location = Preprocessing.substitute_delete_words(test_set.location, data_root)
    test_set.location = Preprocessing.substitute_full_words(test_set.location, data_root)
    
    if "truth" in test_set:
        test_set.truth = Preprocessing.substitute_partial_words(test_set.truth, data_root)
        test_set.truth = Preprocessing.substitute_delete_words(test_set.truth, data_root)
        test_set.truth = Preprocessing.substitute_full_words(test_set.truth, data_root)

In [ ]:
gov = Gov(data_root)
gov.load_data()
old_names = list(gov.ids_by_name.keys())
new_names = Preprocessing.replace_characters_gov(pd.Series(old_names, dtype=str))
new_names = Preprocessing.substitute_partial_words(pd.Series(new_names), data_root)
new_names = Preprocessing.substitute_delete_words(pd.Series(new_names), data_root)
new_names = Preprocessing.substitute_full_words(pd.Series(new_names), data_root)

for old_name, new_name in zip(old_names, new_names):
    gov.ids_by_name[new_name] = gov.ids_by_name[old_name]
    del gov.ids_by_name[old_name]
    
for id_ in gov.names_by_id:
    names = gov.names_by_id[id_]
    new_names =  Preprocessing.replace_characters_gov(pd.Series(names, dtype=str))
    new_names = Preprocessing.substitute_partial_words(pd.Series(new_names), data_root)
    new_names = Preprocessing.substitute_delete_words(pd.Series(new_names), data_root)
    new_names = Preprocessing.substitute_full_words(pd.Series(new_names), data_root)
    
gov.build_indices()

Test Set 1

In [ ]:
for name, test_set in vl_test_sets:
    m = Matcher(gov)
    print("Running", name)
    m.get_match_for_locations(test_set.location)
    total_matches = len([match for match in m.results.values() if match.get("possible_matches")])
    print(f"Total matches: {total_matches} ({round(total_matches / test_set.location.nunique() * 100, 4)}%).")
    print()
    
    result_row.append(total_matches / test_set.location.nunique())

Test Set 2

In [ ]:
for name, test_set in gov_test_sets:
    m = Matcher(gov)
    print("Running", name)
    m.get_match_for_locations(test_set.location)
    total_matches = len([match for match in m.results.values() if match.get("possible_matches")])
    print(f"Total matches: {total_matches} ({round(total_matches / test_set.location.nunique() * 100, 4)}%).")
    
    accuracy = get_accuracy(m.results, test_set)
    print("Accuracy (entries where all parts of truth are in possible matches):", round(accuracy, 4))
    print()
    
    result_row.append(total_matches / test_set.location.nunique())
    result_row.append(accuracy)

Test Set 3

In [ ]:
for name, test_set in syn_test_sets:
    m = Matcher(gov)
    print("Running", name)
    m.get_match_for_locations(test_set.location)
    total_matches = len([match for match in m.results.values() if match.get("possible_matches")])
    print(f"Total matches: {total_matches} ({round(total_matches /  test_set.location.nunique() * 100, 4)}%).")
    
    accuracy = get_accuracy(m.results, test_set)
    print("Accuracy (entries where all parts of truth are in possible matches):", round(accuracy, 4))
    print()
    
    result_row.append(total_matches / test_set.location.nunique())
    result_row.append(accuracy)

Test Set 4

In [ ]:
for name, test_set in gtd_test_sets:
    m = Matcher(gov)
    print("Running", name)
    m.get_match_for_locations(test_set.location)
    total_matches = len([match for match in m.results.values() if match.get("possible_matches")])
    print(f"Total matches: {total_matches} ({round(total_matches /  test_set.location.nunique() * 100, 4)}%).")
    
    accuracy = get_accuracy(m.results, test_set)
    print("Accuracy (entries where all parts of truth are in possible matches):", round(accuracy, 4))
    print()
    
    result_row.append(total_matches / test_set.location.nunique())
    result_row.append(accuracy)

In [ ]:
final_results.append(result_row)

## Auswertung

In [ ]:
names = []
for name, test_set in vl_test_sets + gov_test_sets + syn_test_sets + gtd_test_sets:
    for metric in ['total matches', 'accuracy']:
        if name.startswith("vl") and metric == 'accuracy':
            continue
            
        names.append(name + ' ' + metric)
        
final_results = pd.DataFrame(final_results, columns=names)

In [ ]:
final_results["test set"] = ["Baseline", "Preprocessing VL (corrections + characters)",  "Preprocessing VL + Gov (corrections + characters)",  "Preprocessing VL + Gov (corrections + characters + substitution)"]

In [ ]:
final_results = final_results.set_index("test set")

In [ ]:
final_results.to_csv("final_results.csv")